# Configuring Python Dependencies.

In [2]:
!pip install --upgrade pip
!pip install jax jaxlib
!pip install mediapipe opencv-python

In [3]:
import cv2
import mediapipe as mp
import numpy as np
import math
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

2024-09-23 18:24:34.211305: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-23 18:24:34.213563: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-23 18:24:34.256331: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-23 18:24:34.257297: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-23 18:24:35.025767: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [4]:
def calculate_angle_arctan2(a,b,c):
    """
    arctan2 for calculating the angles between 3 points.
    """
    a = np.array(a) 
    b = np.array(b) 
    c = np.array(c) 
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [5]:
def calculate_angle(a, b, c):
    """
    Calculate the angle between three points a,b and c using
    dot product and cosine angle.
    """
    # Vector BA.
    ba = [a[0] - b[0], a[1] - b[1]]
    # Vector BC.
    bc = [c[0] - b[0], c[1] - b[1]]
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    
    angle = np.arccos(cosine_angle)
    
    return np.degrees(angle)

In [13]:
def calculate_angle_two_points(p1, p2):
    """
    Calculating the angle between two coordinates for 
    Orientation.
    """
    
    ang = math.degrees(np.arctan2(p2[1]-p1[1], p2[0]-p1[0]))

    return ang        

In [16]:
cap = cv2.VideoCapture('Lunge_Alternating.mp4')
w = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
h = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps = cap.get(cv2.CAP_PROP_FPS)

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    
    while cap.isOpened():
        
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        lunges_rep_counter = 0
        
        lunge_right_rep = 0
        lunge_left_rep = 0
        
        # Extract landmarks.
        try:
            
            landmarks = results.pose_landmarks.landmark
            # landmark points.
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            
            # Orientation landmarks.
            nose = [landmarks[mp_pose.PoseLandmark.NOSE.value].x,landmarks[mp_pose.PoseLandmark.NOSE.value].y]
            left_eye = [landmarks[mp_pose.PoseLandmark.LEFT_EYE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_EYE.value].y]
            right_eye = [landmarks[mp_pose.PoseLandmark.RIGHT_EYE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_EYE.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            
            shoulder_midpoint = [(left_shoulder[0] + right_shoulder[0]) / 2, (left_shoulder[1] + right_shoulder[1])/2]
            
            # Orientation angles.
            eye_angle = calculate_angle_two_points(left_eye, right_eye)
            # Orientation angle of nose and the midpoint of the shoulders.
            nose_shoulder_angle = calculate_angle_two_points(nose, shoulder_midpoint)
            
            # Calculate angles
            left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
            right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
            left_hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
            right_hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)
            
            # Calculate step length
            #print(right_ankle)
            #print(left_ankle)
            ankle_step_length = math.dist(left_ankle,right_ankle)
            knee_step_length = math.dist(left_knee,right_knee)
            #print(step_length)
            #print(left_knee_angle)
            
            str_eye_angle = f"EYE_ANGLE: {eye_angle}"
            
            str_nose_shoulder_angle = f"NOSE_SHOULDER_ANGLE: {nose_shoulder_angle}"
            
            is_left_lunge_good = "FALSE"
            is_right_lunge_good = "FALSE"
            
            str_left_knee_angle = f"LEFT_KNEE_ANGLE: {left_knee_angle}"
            
            str_right_knee_angle = f"RIGHT_KNEE_ANGLE: {right_knee_angle}"
            
            str_left_hip_angle = f"LEFT_HIP_ANGLE: {left_hip_angle}"
            
            str_right_hip_angle = f"RIGHT_HIP_ANGLE: {right_hip_angle}"
            
            str_right_hip_angle = f"RIGHT_HIP_ANGLE: {right_hip_angle}"
            
            str_ankle_step_length = f"ANKLE_STEP_LENGTH: {ankle_step_length}"
            
            str_knee_step_length = f"KNEE_STEP_LENGTH: {knee_step_length}"
            
            #cv2.putText(image, str_knee_step_length,(50, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (100, 50, 0), 1, cv2.LINE_AA)
                
            #cv2.putText(image, str_ankle_step_length,(50, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (100, 50, 0), 1, cv2.LINE_AA)
            
            #cv2.putText(image, str_left_knee_angle,(50, 200), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (100, 50, 0), 1, cv2.LINE_AA)
            
            #cv2.putText(image, str_right_knee_angle,(50, 250), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (100, 50, 0), 1, cv2.LINE_AA)
            
            #cv2.putText(image, str_left_hip_angle,(50, 300), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (100, 50, 0), 1, cv2.LINE_AA)
            
            #cv2.putText(image, str_right_hip_angle,(50, 350), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (100, 50, 0), 1, cv2.LINE_AA)
            
            #cv2.putText(image, str_eye_angle,(50, 500), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (100, 50, 0), 1, cv2.LINE_AA)
            
            #cv2.putText(image, str_nose_shoulder_angle,(50, 550), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (100, 50, 0), 1, cv2.LINE_AA)
            
            
            if nose_shoulder_angle < 80 and (eye_angle > 150 or eye_angle < 0):
                
                    print("Perpendicular VIEW")
                    
                    if (80 <= left_knee_angle <= 90) and (85 <= left_hip_angle <= 100) and (ankle_step_length >= 0.14) and (knee_step_length > 0.15):
                        
                        print("Right KNEE")
                        print(right_knee)
                        is_left_lunge_good = "TRUE"
                        
                        lunge_right_rep = +1
            
                    if (70 <= right_knee_angle <= 90) and (80 <= right_hip_angle <= 100) and (ankle_step_length >= 0.14) and (knee_step_length > 0.18):
                        
                        print("LEFT KNEE")
                        print(left_knee)
                        is_right_lunge_good = "TRUE"
                        
                        lunge_left_rep = +1
                        
            if (80 > nose_shoulder_angle >= 70) and (eye_angle > 150 or eye_angle < 0):
                
                    print("Angular VIEW")
                    
                    if (100 <= left_knee_angle <= 130) and (100 <= left_hip_angle <= 130) and (ankle_step_length >= 0.11) and (knee_step_length > 0.06):
                            
                        print("Right KNEE")
                        print(right_knee)
                        is_left_lunge_good = "TRUE"
                        lunge_left_rep = +1
                    
                    if (100 <= right_knee_angle <= 130) and (100 <= right_hip_angle <= 130) and (ankle_step_length >= 0.16) and (knee_step_length >= 0.18):
                        
                        print("LEFT KNEE")
                        print(left_knee)
                        is_right_lunge_good = "TRUE"
                        lunge_right_rep = +1 
                        
            elif nose_shoulder_angle > 80 and eye_angle > 150:
                    
                    print("FRONTAL VIEW")
                    
                    if (0 < right_knee_angle <= 10) and (150 <= left_knee_angle <= 180) and (ankle_step_length >= 0.1) and (knee_step_length >= 0.1):
                        
                        print("LEFT KNEE")
                        print(left_knee)
                        is_left_lunge_good = "TRUE"
                        lunge_left_rep = +1
                    
                    if (0 < left_knee_angle <= 10) and (150 <= right_knee_angle <= 180) and (ankle_step_length >= 0.1) and (knee_step_length >= 0.1):
                        
                        print("RIGHT KNEE")
                        print(right_knee)
                        is_right_lunge_good = "TRUE"
                        lunge_right_rep = +1
            
            str_is_left_lunge_good = "IS_LEFT_LUNGE_GOOD: " + is_left_lunge_good
            str_is_right_lunge_good = "IS_RIGHT_LUNGE_GOOD: " + is_right_lunge_good
            
            str_is_left_lunge_count = "LEFT_LUNGE_COUNTER :- {}".format(lunge_left_rep)
            str_is_right_lunge_count = "RIGHT_LUNGE_COUNTER:- {}".format(lunge_right_rep)
            
            
            cv2.putText(image, str_is_left_lunge_good,(50, 200), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (100, 50, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str_is_right_lunge_good,(50, 250), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (100, 50, 0), 1, cv2.LINE_AA)
            
            cv2.putText(image, str_is_left_lunge_count,(50, 300), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (100, 50, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str_is_right_lunge_count,(50, 350), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (100, 50, 0), 1, cv2.LINE_AA)
            
        except:
            
            print("Not able to process image frames:-")
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))               
        
        cv2.imshow('Mediapipe Lunge Analyzer.', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open zink: /usr/lib/dri/zink_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: 

Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process 

Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Perpendicular VIEW
Angular VIEW
Perpendicular VIEW
Angular VIEW
Perpendicular VIEW
Angular VIEW
Perpendicular VIEW
Angular VIEW
Perpendicular VIEW
Angular VIEW
Perpendicular VIEW
Angular VIEW
Perpendicu

Perpendicular VIEW
Angular VIEW
Right KNEE
[0.5057200193405151, 0.7093350887298584]
Perpendicular VIEW
Angular VIEW
Perpendicular VIEW
Angular VIEW
Perpendicular VIEW
Angular VIEW
Perpendicular VIEW
Angular VIEW
Perpendicular VIEW
Angular VIEW
Perpendicular VIEW
Angular VIEW
Perpendicular VIEW
Angular VIEW
Perpendicular VIEW
Angular VIEW
Perpendicular VIEW
Angular VIEW
Perpendicular VIEW
Angular VIEW
Perpendicular VIEW
Angular VIEW
Perpendicular VIEW
Angular VIEW
Perpendicular VIEW
Angular VIEW
Perpendicular VIEW
Angular VIEW
Perpendicular VIEW
Angular VIEW
Perpendicular VIEW
Angular VIEW
Perpendicular VIEW
Angular VIEW
Perpendicular VIEW
Angular VIEW
Perpendicular VIEW
Angular VIEW
Perpendicular VIEW
Angular VIEW
Perpendicular VIEW
Angular VIEW
Perpendicular VIEW
Angular VIEW
Perpendicular VIEW
Angular VIEW
Perpendicular VIEW
Angular VIEW
Perpendicular VIEW
Angular VIEW
Perpendicular VIEW
Angular VIEW
Perpendicular VIEW
Angular VIEW
Perpendicular VIEW
Angular VIEW
Perpendicular VIEW
A

Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process image frames:-
Not able to process 